In [1]:
import os
import pandas as pd
import statsmodels.api as sm

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest, f_classif

In [3]:
market = pd.read_csv(os.path.join('resources', 'cleaned_data.csv')).drop(columns=['Unnamed: 0'])

In [ ]:
# indicators = pd.DataFrame({
#    'Indicator': market.columns.tolist()
# })

# filt = indicators[indicators['Indicator'].str.contains('Growth|Margin|ratio|Ratio|Turnover|per|/| / | to |Per|return|To|days') == False]

# cond = filt['Indicator'].to_list()

In [29]:
y = market['Class']

In [30]:
x = market.drop(columns={'Ticker','Company Name','2019 PRICE VAR [%]','Class','Market Cap Size'})

X = pd.get_dummies(x)

X.head()

,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,...,Sector_Communication Services,Sector_Consumer Cyclical,Sector_Consumer Defensive,Sector_Energy,Sector_Financial Services,Sector_Healthcare,Sector_Industrials,Sector_Real Estate,Sector_Technology,Sector_Utilities
0,15926000.0,-0.1230,0.0,15926000.0,0.0,22534000.0,22534000.0,-6608000.0,0.0,-10501000.0,...,0,0,0,0,1,0,0,0,0,0
1,185553000.0,0.0245,132650000.0,52903000.0,0.0,39042000.0,51837000.0,1066000.0,2037000.0,-971000.0,...,0,0,0,0,1,0,0,0,0,0
2,267465000.0,-0.3004,179692000.0,87773000.0,49903000.0,48638000.0,98541000.0,-10768000.0,1106000.0,-9778000.0,...,0,0,0,0,0,0,0,0,1,0
3,433947000.0,0.0709,330414000.0,103533000.0,0.0,47755000.0,47743000.0,55790000.0,0.0,55939000.0,...,0,0,0,0,0,0,0,0,0,0
4,330867000.0,0.0504,121455000.0,209412000.0,0.0,22784000.0,129877000.0,79535000.0,52248000.0,19686000.0,...,0,0,0,0,0,0,0,1,0,0


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

classifier = LogisticRegression()

classifier.fit(X_train, y_train)

print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

/Users/danvaldes/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Training Data Score: 0.691559569294894
Testing Data Score: 0.6833333333333333


/Users/danvaldes/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [32]:
y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.45      0.09      0.16       298
           1       0.70      0.95      0.81       662

    accuracy                           0.68       960
   macro avg       0.58      0.52      0.48       960
weighted avg       0.62      0.68      0.60       960



In [6]:
selector = SelectKBest(f_classif, k=12)

X_new = selector.fit_transform(X, y)

selected_features = pd.DataFrame(selector.inverse_transform(X_new), 
                                 index=X.index, 
                                 columns=X.columns)
selected_features.head()

# Dropped columns have values of all 0s, so var is 0, drop them
selected_columns = selected_features.columns[selected_features.var() != 0]

# Get the valid dataset with the selected features.
X_final = selected_features[selected_columns]

In [26]:
X_final.head()

,Operating Income,Earnings before Tax,Net Income,Net Income Com,Dividend per Share,EBIT,Consolidated Income,Market Cap,Enterprise Value,5Y Dividend per Share Growth (per Share),Sector_Financial Services,Sector_Healthcare
0,-6608000.0,-10501000.0,-10876000.0,-11082000.0,0.00,-10501000.0,-10876000.0,4.848362e+07,6.546622e+06,0.0000,1.0,0.0
1,1066000.0,-971000.0,-704000.0,-1103000.0,0.02,1066000.0,-704000.0,4.868035e+07,6.978835e+07,0.0000,1.0,0.0
2,-10768000.0,-9778000.0,-2146000.0,-2146000.0,0.00,-8672000.0,-2146000.0,3.054934e+08,3.360344e+08,0.0000,0.0,0.0
3,55790000.0,55939000.0,42572000.0,42572000.0,0.32,55939000.0,42572000.0,1.826434e+09,1.824413e+09,0.2358,0.0,0.0
4,79535000.0,19686000.0,19686000.0,19686000.0,1.09,71934000.0,27202000.0,1.896913e+09,3.130413e+09,0.0510,0.0,0.0


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y, random_state=1, stratify=y)

classifier = LogisticRegression()

classifier.fit(X_train, y_train)

print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

In [28]:
y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.55      0.12      0.20       298
           1       0.71      0.95      0.81       662

    accuracy                           0.70       960
   macro avg       0.63      0.54      0.51       960
weighted avg       0.66      0.70      0.62       960

